In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
    

In [2]:
ml_client = MLClient.from_config(credential= credential)

Found the config file in: .\config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [8]:
%%writefile inputs/sorvetes-training.py

import pandas as pd
import  numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import mlflow

with mlflow.start_run():
    data = 'vendas_sorvete_dataset.csv'
    sorvetes = pd.read_csv(data)

    x, y = sorvetes[['temperatura']].values, sorvetes['vendas'].values

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

    model = LinearRegression().fit(x_train, y_train)

    y_hat = model.predict(x_test)

    mse = mean_squared_error(y_test, y_hat)
    rmse = np.sqrt(mse)                         #taking the square root to improve understanding
    r2 = r2_score(y_test, y_hat)

    print(f'RMSE: {rmse:.2f}')
    print(f'R^2: {r2:.4f}')

    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("R2_score", r2)

    mlflow.sklearn.log_model(model, "ice_cream_model")



Writing inputs/sorvetes-training.py


In [3]:
from azure.ai.ml import command

job = command(
    code='./inputs',
    command='python sorvetes-training.py',
    environment='AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest',
    compute='cpu-cluster',
    display_name='My-first-model-LinReg',
    experiment_name='sales-training'
)

config_job = ml_client.create_or_update(job)
aml_url = config_job.studio_url
print(f'URL: {aml_url}')

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


URL: https://ml.azure.com/runs/boring_shark_vzkzgpsn6w?wsid=/subscriptions/5ccc7bcb-0174-494b-9b6e-7381826b8945/resourcegroups/rg-projeto-1/workspaces/projeto1WS&tid=b7c7f126-3267-43cc-9c64-e95984dd500f


In [ ]:
from azure.ai.ml.entities import Model

job_name = config_job.name

model_path = f'runs:/{job_name}/ice_cream_model'

reg_model = ml_client.models.create_or_update(
    Model(
        name='ice-cream-sales-model',
        version='1',
        path=model_path,
        type='mlflow_model'
    )
)

HttpResponseError: (UserError) The request is invalid.
Code: UserError
Message: The request is invalid.
Exception Details:	(NoMatchingArtifactsFoundFromRun) No artifacts matching ice_cream_model found from Run.
	Code: NoMatchingArtifactsFoundFromRun
	Message: No artifacts matching ice_cream_model found from Run.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "4111ac7c73eb7e4a09ff46b25a47789f",
        "request": "42a45ebec518b623"
    }
}Type: Environment
Info: {
    "value": "brazilsouth"
}Type: Location
Info: {
    "value": "brazilsouth"
}Type: Time
Info: {
    "value": "2025-10-31T23:43:43.161962+00:00"
}

In [16]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment
import uuid

endpoint_name = f'ice-cream-sales-{uuid.uuid4().hex[:6]}'

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode='key',
    description='Predicts ice cream sales based on temperature.'
)

deployment = ManagedOnlineDeployment(
    name='blue',
    endpoint_name=endpoint_name,
    model=reg_model.id,
    instance_type='Standard_F2s_v2',
    instance_count=1
)

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()

ml_client.online_deployments.begin_create_or_update(deployment).wait()

endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()

In [8]:
import json

test_data_path = "sample_request.json"

test_json = {
    "input_data": [
        [30.5],
        [15.0]
    ]
}

with open(test_data_path, "w") as f:
    json.dump(test_json, f)

predictions = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file=test_data_path,
)

print("Predictions received:")
print(predictions)

NameError: name 'endpoint_name' is not defined

#### Be cautios, this cell deletes the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name).wait()